# 4. Inference

In [1]:
import sys, os
import torch
import pandas as pd
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")
import setting

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import scaler as ml_scaler
from Clust.clust.ML.tool import clean as ml_clean
import pathSetting

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


In [2]:
# Set regression mode i.e., 'regression','forecast'
mode_selection = "forecast"

# get integrated data name
bucket_name = 'integration'

# scaler path
scalerPath = './scaler/'

### 4-1. Get model meta by mongodb

In [3]:
db_name = 'model'
collection_name = 'meta'
search = {'modelName': 'forecasting_Hs2SwineFarmWithWeatherTime_LSTM_rg_NoClean'}

model_meta= mongo_client.get_document_by_json(db_name, collection_name, search)[0]
model_meta

{'trainDataInfo': {'data_name': 'forecasting_Hs2SwineFarmWithWeatherTime_train_NoClean',
  'ingestion_param': {'start_time': '2021-02-01 00:00:00',
   'end_time': '2021-03-10 00:00:00',
   'ms_list_info': [['farm_swine_air', 'HS2'],
    ['weather_outdoor_keti_clean', 'sangju'],
    ['life_additional_Info', 'trigonometicInfoByHours']]},
  'integration_param': {'param': {},
   'method': 'meta',
   'integration_duration': 'common',
   'integration_frequency': 300},
  'clean_param': 'NoClean',
  'process_param': {'refine_param': {'removeDuplication': {'flag': False},
    'staticFrequency': {'flag': False, 'frequency': None}},
   'outlier_param': {'certainErrorToNaN': {'flag': False},
    'unCertainErrorToNaN': {'flag': False, 'param': {}}},
   'imputation_param': {'flag': False,
    'imputation_method': [],
    'totalNonNanRatio': 80}}},
 'modelName': 'forecasting_Hs2SwineFarmWithWeatherTime_LSTM_rg_NoClean',
 'featureList': ['Temperature', 'out_temp', 'sin_hour'],
 'target': 'Temperature'

### 4-2. Inference data selection

In [4]:
# 0. pick only one data
collection_list = mongo_client.get_collection_list(bucket_name)
collection_list

['forecasting_Hs2SwineFarmWithWeatherTime',
 'regression_energy',
 'forecasting_strawberryOpen',
 'forecasting_gunwiStrawberryWeather']

In [5]:
if mode_selection == 'regression':

    data_name_X = 'regression_energy_testX_NoClean'
    dataX = db_client.get_data(bucket_name, data_name_X)
    
    cleanMode = model_meta["trainDataInfo"]["clean_param"]
    integration_freq_sec = model_meta["trainDataInfo"]["integration_param"]["integration_frequency"]

    
elif mode_selection == 'forecast':
    
    data_name_X = 'forecasting_Hs2SwineFarmWithWeatherTime_test_NoClean'
    dataX = db_client.get_data(bucket_name, data_name_X)
    
    cleanMode = model_meta["trainDataInfo"]["clean_param"]
    integration_freq_sec = model_meta["trainDataInfo"]["integration_param"]["integration_frequency"]

In [6]:
print(dataX.head)

<bound method NDFrame.head of                                 CO2ppm    H2Sppm   Humidity      NH3ppm  \
time                                                                      
2021-03-10 00:00:00+00:00  3991.666667  3.000000  64.866667  142.000000   
2021-03-10 00:05:00+00:00  4039.500000  3.000000  65.000000  142.000000   
2021-03-10 00:10:00+00:00  4066.000000  3.666667  64.966667  143.666667   
2021-03-10 00:15:00+00:00  4046.500000  3.500000  64.750000  144.000000   
2021-03-10 00:20:00+00:00  4036.000000  3.333333  64.433333  144.666667   
...                                ...       ...        ...         ...   
2021-03-16 22:40:00+00:00  3278.333333  3.333333  62.733333  128.666667   
2021-03-16 22:45:00+00:00  3290.500000  3.000000  62.800000  128.500000   
2021-03-16 22:50:00+00:00  3290.333333  3.333333  62.633333  127.666667   
2021-03-16 22:55:00+00:00  3285.000000  3.500000  62.300000  126.500000   
2021-03-16 23:00:00+00:00  3263.000000  3.000000  62.000000  127.00000

### 4-3. Inference data preparation

In [7]:
feature_list = model_meta['featureList']
target = model_meta['target']
scaler_param = model_meta['scalerParam']
transform_param = model_meta["transformParameter"]
model_file_path = model_meta['files']['modelFile']['filePath']
model_method = model_meta['model_method']
# train_parameter = model_meta["trainParameter"]

if mode_selection == 'regression':
    # Scaling Inference Input
    X_scaler_file_path = model_meta['files']['XScalerFile']["filePath"]
    y_scaler_file_path = model_meta['files']['yScalerFile']["filePath"]

    infer_X, scaler_X = ml_scaler.get_scaled_test_data(dataX[feature_list], X_scaler_file_path, scaler_param)
    scaler_y = ml_scaler.get_scaler_file(y_scaler_file_path)

elif mode_selection == 'forecast':
    clean_param = model_meta['cleanTrainDataParam']
    nan_processing_param = model_meta['NaNProcessingParam']

    # Scaling Inference Input
    X_scaler_file_path = model_meta['files']['XScalerFile']['filePath']

    # only for forecast data?
    past_step = transform_param['past_step']
    feature_data = dataX[feature_list]
    step_data = feature_data[-past_step:][feature_list].values
    df_data = pd.DataFrame(step_data, columns=feature_list)

    infer_X, scaler_X = ml_scaler.get_scaled_test_data(df_data[feature_list], X_scaler_file_path, scaler_param)
    clean_infer_X = ml_clean.get_cleand_data(infer_X, clean_param, integration_freq_sec, nan_processing_param)

In [14]:
print(infer_X)

    Temperature  out_temp  sin_hour
0     -0.072961  0.429947 -0.487796
1     -0.072961  0.419251 -0.475592
2     -0.072961  0.408556 -0.463388
3     -0.068670  0.397861 -0.451184
4     -0.060086  0.387166 -0.438981
5     -0.064378  0.376471 -0.426777
6     -0.072961  0.365775 -0.414573
7     -0.064378  0.355080 -0.402369
8     -0.060086  0.344385 -0.390165
9     -0.030043  0.333690 -0.377961
10    -0.002146  0.322995 -0.365757
11    -0.008584  0.312299 -0.353553
12     0.023605  0.303743 -0.339342
13     0.060086  0.295187 -0.325130
14     0.100858  0.286631 -0.310918
15     0.133047  0.278075 -0.296706
16     0.165236  0.269519 -0.282495
17     0.188841  0.260963 -0.268283
18     0.197425  0.252406 -0.254071
19     0.214592  0.243850 -0.239860
20     0.236052  0.235294 -0.225648
21     0.248927  0.226738 -0.211436
22     0.261803  0.218182 -0.197224
23     0.261803  0.209626 -0.183013


In [9]:
inferX = infer_X
# # transform
# if mode_selection == 'regression':

#     from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf
#     inferX = trans_df_to_np_inf(infer_X)

# # forecast
# elif mode_selection == 'forecast':
    
#     from Clust.clust.transformation.purpose.machineLearning import LSTMData
#     LSTMD = LSTMData()
#     testX, testy = LSTMD.transform_Xy_arr(clean_infer_X, transformParameter, transformParameter['clean_param'])

In [10]:
print(type(inferX), inferX.shape)
print(inferX)

<class 'pandas.core.frame.DataFrame'> (24, 3)
    Temperature  out_temp  sin_hour
0     -0.072961  0.429947 -0.487796
1     -0.072961  0.419251 -0.475592
2     -0.072961  0.408556 -0.463388
3     -0.068670  0.397861 -0.451184
4     -0.060086  0.387166 -0.438981
5     -0.064378  0.376471 -0.426777
6     -0.072961  0.365775 -0.414573
7     -0.064378  0.355080 -0.402369
8     -0.060086  0.344385 -0.390165
9     -0.030043  0.333690 -0.377961
10    -0.002146  0.322995 -0.365757
11    -0.008584  0.312299 -0.353553
12     0.023605  0.303743 -0.339342
13     0.060086  0.295187 -0.325130
14     0.100858  0.286631 -0.310918
15     0.133047  0.278075 -0.296706
16     0.165236  0.269519 -0.282495
17     0.188841  0.260963 -0.268283
18     0.197425  0.252406 -0.254071
19     0.214592  0.243850 -0.239860
20     0.236052  0.235294 -0.225648
21     0.248927  0.226738 -0.211436
22     0.261803  0.218182 -0.197224
23     0.261803  0.209626 -0.183013


In [11]:
modelParameter = model_meta["modelParameter"]

inferParameter = {
    'device': 'cpu',
    'batch_size': 16
}

In [12]:
# 4. Inference
from Clust.clust.ML.regression_YK.inference import RegressionInference as RI

ri = RI()
ri.set_param(inferParameter)
ri.set_model(model_method, model_file_path, modelParameter)
ri.set_data(inferX)
preds = ri.inference()
print(preds)

ValueError: could not determine the shape of object type 'DataFrame'

In [ ]:
# for regression
if mode_selection == 'regression':
    if scaler_param =='scale':
        base_df_for_inverse = pd.DataFrame(columns=target, index=range(len(preds)))
        base_df_for_inverse[target] = preds
        prediction_result = pd.DataFrame(scaler_y.inverse_transform(base_df_for_inverse), columns=target, index=base_df_for_inverse.index)
    else:
        prediction_result = pd.DataFrame(data={"value": preds}, index=range(len(preds)))

# for forecast
elif mode_selection == 'forecast':
    if scaler_param =='scale':
        base_df_for_inverse = pd.DataFrame(columns=feature_list, index=range(len(preds)))
        base_df_for_inverse[target] = preds
        inverse_result = pd.DataFrame(scaler_X.inverse_transform(base_df_for_inverse), columns=feature_list, index=base_df_for_inverse.index)
        target_data = inverse_result[target]
        prediction_result = pd.DataFrame(data={target: target_data}, index=range(len(preds)))
    else:
        prediction_result = pd.DataFrame(data={target: preds}, index=range(len(preds)))

In [ ]:
prediction_result